In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, Literal
from langchain_openai import ChatOpenAI
from langsmith import evaluate
from patsy import state
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import operator
from langchain_core.messages import SystemMessage, HumanMessage




C:\Users\Aritri Baidya\Desktop\MyFiles\Pycharm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()

generator_llm= ChatOpenAI(model='gpt-4o')
evaluation_llm= ChatOpenAI(model='gpt-4o-mini')
optimizer_llm= ChatOpenAI(model='gpt-4o')

In [ ]:
#state
class TweetState(TypedDict):
    topic: str
    tweet: str
    evaluation: Literal['approved', 'needs_improvement']
    feedback: str
    iteration: int
    max_iteration: int

    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

In [ ]:
class TweetEvaluation(BaseModel):
    evaluation: Literal['approved', 'needs_improvement']= Field(...,description='Final evaluation of the tweet')
    feedback: str = Field(...,description='Constructive feedback for the tweet.')


In [ ]:
structured_evaluator_llm=evaluation_llm.with_structured_output(TweetEvaluation)

In [ ]:
def generate_tweet(state: TweetState):
    #prompt
    messages= [
        SystemMessage(content='You are a funny and clever Twitter/X influencer'),
        HumanMessage(content=f"""
Write a short, original and hilarious tweet on the topic: "{state['topic']}" .
Rules:
- Do not use question-answer format.
- Max 200 characters.
-Use observational humor, irony, sarcasm, or cultural references.
-Think in meme logic, punchlines, or reliable takes.
-Use simple, day to day english
-This is version {state['iteration']}""")
    ]


    # send to generator_llm
    response=generator_llm.invoke(messages).content

    #return response
    return {'tweet': response, 'tweet_history': [response]}


In [ ]:
def evaluate_tweet(state: TweetState):
    #prompt
    messages= [
        SystemMessage(content='You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets.'),
        HumanMessage(content=f"""
Evaluate the following tweet:
Tweet: "{state['tweet']}" .
Use the criteria below to evaluate the tweet:
1. Originality: Is this fresh , or have you seen it a hundred times before?
2. Humor: Did it genuinely make you smile, laugh, or chunckle?
3. Punchiness: Is it short, sharp, and scroll-stopping?
4. Virality Potential: Would people retweet or share it?
5. Format: Is it a well-formed tweet (not a setup-puchline joke, not a Q&A joke, and under or equal to 200 characters.)

Auto-reject if:
-Its written in question-answer format (e.g., "Wht did.." or "What happens when..")
-It exceeds 200 characters
-It reads like a traditional setup-punchline joke
-Do not end with generic, throwaway, or deflating lines that weaken the humor.
###Respond ONLY in structured format:
-evaluation: 'approved' or 'needs_improvement'
-feedback: One paragraph explaining the strengths and weaknesses """)

    ]
    response=structured_evaluator_llm.invoke(messages)

    return {'evaluation': response.evaluation, 'feedback': response.feedback, 'feedback_history': [response.feedback]}


In [ ]:
def optimize_tweet(state: TweetState):
    messages=[
        SystemMessage(content='You punch up tweets for virality and humor based on given feedback.'),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
'{state['feedback']}'
Topic: "{state['topic']}"
Original Tweet: "{state['tweet']}"
Rewrite it as a short, viral-worthy tweet. Avoid Q&A style and stay under or equal to 200 characters""")
    ]
    response=optimizer_llm.invoke(messages).content
    iteration= state['iteration'] +1

    return {'tweet': response, 'iteration': iteration, 'tweet_history': [response]}

In [ ]:
#function that controls looping
def route_evaluation(state: TweetState):
    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'


In [ ]:
graph=StateGraph(TweetState)

graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')


graph.add_conditional_edges('evaluate', route_evaluation, {'approved': END, 'needs_improvement': 'optimize_tweet'})
graph.add_edge('optimize', 'evaluate')

#compile
workflow=graph.compile()

In [ ]:
#to see the workflow
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png())

In [ ]:
initial_state={
    'topic': 'Indian Railways',
    'iteration': 1,
    'max_iteration': 5
}
final_state=workflow.invoke(initial_state)
print(final_state)